**Importando as bibliotecas**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

**Coletando os dados web**

In [ ]:
def scrap_state_info(state: str) -> dict:

  print(f'  Picking {state} info ...')
  state_url = f'https://ibge.gov.br/cidades-e-estados/{state}.html'
  page = requests.get(state_url)

  soup = BeautifulSoup(page.content, 'html.parser')
  indicadors = soup.select('.indicador')

  
  state_dict = {  
       ind.select('.ind-label')[0].text: ind.select('.ind-value')[0].text 
       for ind in indicadors
      }


  return state_dict 

  ##return [(ind.select('.ind-label')[0].text, ind.select('.ind-value')[0].text) for ind in indicadors]





In [ ]:
states = ['RS', 'SC','PR','SP','RJ','MG','ES','MT','MS','DF','GO','AM','BA','CE','PA','PB','PE','PI','RN','RO','RR','SE','TO','MA']

states_data = [scrap_state_info(state)for state in states]

  Picking RS info ...
  Picking SC info ...
  Picking PR info ...
  Picking SP info ...
  Picking RJ info ...
  Picking MG info ...
  Picking ES info ...
  Picking MT info ...
  Picking MS info ...
  Picking DF info ...
  Picking GO info ...
  Picking AM info ...
  Picking BA info ...
  Picking CE info ...
  Picking PA info ...
  Picking PB info ...
  Picking PE info ...
  Picking PI info ...
  Picking RN info ...
  Picking RO info ...
  Picking RR info ...
  Picking SE info ...
  Picking TO info ...
  Picking MA info ...


In [ ]:
df = pd.DataFrame(states_data)

df.head()

,Governador,Capital,Gentílico,Área Territorial,População estimada,Densidade demográfica,Matrículas no ensino fundamental,IDH Índice de desenvolvimento humano,Receitas realizadas,Despesas empenhadas,Rendimento mensal domiciliar per capita,Total de veículos
0,EDUARDO FIGUEIREDO CAVALHEIRO LEITE [2019],Porto Alegre [2010],gaúcho ou sul-rio-grandense,"281.707,149 km² [2020]",11.422.973 pessoas [2020],"39,79 hab/km² [2010]",1.269.309 matrículas [2020],"0,746 [2010]","66.397.468,18 R$ (×1000) [2017]","62.476.279,34 R$ (×1000) [2017]",1.759 R$ [2020],7.495.615 veículos [2020]
1,CARLOS MOISÉS DA SILVA [2019],Florianópolis [2010],catarinense ou barriga-verde,"95.730,684 km² [2020]",7.252.502 pessoas [2020],"65,29 hab/km² [2010]",876.392 matrículas [2020],"0,774 [2010]","34.696.772,82 R$ (×1000) [2017]","25.595.103,38 R$ (×1000) [2017]",1.632 R$ [2020],5.583.126 veículos [2020]
2,CARLOS ROBERTO MASSA JUNIOR [2019],Curitiba [2010],paranaense,"199.298,982 km² [2020]",11.516.840 pessoas [2020],"52,40 hab/km² [2010]",1.407.978 matrículas [2020],"0,749 [2010]","60.163.576,12 R$ (×1000) [2017]","55.534.402,97 R$ (×1000) [2017]",1.508 R$ [2020],8.077.413 veículos [2020]
3,JOÃO AGRIPINO DA COSTA DORIA JUNIOR [2019],São Paulo [2010],paulista,"248.219,481 km² [2020]",46.289.333 pessoas [2020],"166,25 hab/km² [2010]",5.414.208 matrículas [2020],"0,783 [2010]","232.822.496,57 R$ (×1000) [2017]","231.982.243,69 R$ (×1000) [2017]",1.814 R$ [2020],30.778.960 veículos [2020]
4,CLÁUDIO BOMFIM DE CASTRO E SILVA [2019],Rio de Janeiro [2010],fluminense,"43.750,426 km² [2020]",17.366.189 pessoas [2020],"365,23 hab/km² [2010]",1.967.998 matrículas [2020],"0,761 [2010]","78.488.140,79 R$ (×1000) [2017]","67.965.548,70 R$ (×1000) [2017]",1.723 R$ [2020],7.087.797 veículos [2020]


**Realizando uma cópia do DataFrame**

In [ ]:
states_df = df.copy()

In [ ]:
states_df.columns

Index(['Governador', 'Capital', 'Gentílico', 'Área Territorial',
       'População estimada', 'Densidade demográfica',
       'Matrículas no ensino fundamental',
       'IDH Índice de desenvolvimento humano', 'Receitas realizadas',
       'Despesas empenhadas', 'Rendimento mensal domiciliar per capita',
       'Total de veículos'],
      dtype='object')

LIMPEZA DOS DADOS. LIMPANDO VÍRGULAS, BARRAS ETC...

In [ ]:
states_df = states_df.replace({
    '\.': '',
     ',': '.',
     '\[\d+\]': '',
     'hab/km²': '',
     'km²': '',
     ' pessoas': '',
     ' matrículas': '',
     'R\$.*': '',
     ' veículos': ''
}, regex =True)

states_df.head()

,Governador,Capital,Gentílico,Área Territorial,População estimada,Densidade demográfica,Matrículas no ensino fundamental,IDH Índice de desenvolvimento humano,Receitas realizadas,Despesas empenhadas,Rendimento mensal domiciliar per capita,Total de veículos
0,EDUARDO FIGUEIREDO CAVALHEIRO LEITE,Porto Alegre,gaúcho ou sul-rio-grandense,281707149,11422973,3979,1269309,0746,6639746818,6247627934,1759,7495615
1,CARLOS MOISÉS DA SILVA,Florianópolis,catarinense ou barriga-verde,95730684,7252502,6529,876392,0774,3469677282,2559510338,1632,5583126
2,CARLOS ROBERTO MASSA JUNIOR,Curitiba,paranaense,199298982,11516840,5240,1407978,0749,6016357612,5553440297,1508,8077413
3,JOÃO AGRIPINO DA COSTA DORIA JUNIOR,São Paulo,paulista,248219481,46289333,16625,5414208,0783,23282249657,23198224369,1814,30778960
4,CLÁUDIO BOMFIM DE CASTRO E SILVA,Rio de Janeiro,fluminense,43750426,17366189,36523,1967998,0761,7848814079,6796554870,1723,7087797


**Dropando as colunas irrelevantes**

In [ ]:
states_df.drop('Total de veículos', axis=1, inplace=True)

In [ ]:
states_df.drop('Densidade demográfica', axis=1, inplace=True)

**Limpando as colunas númericos como tipo númerico**

In [ ]:
num_cols = ['Área Territorial', 'População estimada', 'Matrículas no ensino fundamental', 'IDH Índice de desenvolvimento humano', 'Receitas realizadas', 'Despesas empenhadas', 'Rendimento mensal domiciliar per capita']

In [ ]:
states_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 9 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   Governador                               24 non-null     object
 1   Capital                                  24 non-null     object
 2   Área Territorial                         24 non-null     int64 
 3   População estimada                       24 non-null     int64 
 4   Matrículas no ensino fundamental        24 non-null     int64 
 5   IDH Índice de desenvolvimento humano    24 non-null     int64 
 6   Receitas realizadas                      24 non-null     int64 
 7   Despesas empenhadas                      24 non-null     int64 
 8   Rendimento mensal domiciliar per capita  24 non-null     int64 
dtypes: int64(7), object(2)
memory usage: 1.8+ KB


In [ ]:
##states_df[num_cols] = states_df[num_cols].apply(lambda x: x.str.strip())
##states_df[num_cols] = states_df[num_cols].apply(pd.to_numeric)

In [ ]:
states_df.head()

,Governador,Capital,Área Territorial,População estimada,Matrículas no ensino fundamental,IDH Índice de desenvolvimento humano,Receitas realizadas,Despesas empenhadas,Rendimento mensal domiciliar per capita
0,EDUARDO FIGUEIREDO CAVALHEIRO LEITE,Porto Alegre,281707149,11422973,1269309,746,6639746818,6247627934,1759
1,CARLOS MOISÉS DA SILVA,Florianópolis,95730684,7252502,876392,774,3469677282,2559510338,1632
2,CARLOS ROBERTO MASSA JUNIOR,Curitiba,199298982,11516840,1407978,749,6016357612,5553440297,1508
3,JOÃO AGRIPINO DA COSTA DORIA JUNIOR,São Paulo,248219481,46289333,5414208,783,23282249657,23198224369,1814
4,CLÁUDIO BOMFIM DE CASTRO E SILVA,Rio de Janeiro,43750426,17366189,1967998,761,7848814079,6796554870,1723


**Exportando o DataFrame em formato de CSV para o Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

states_df.to_csv('/content/drive/MyDrive/Colab Notebooks/ibge_states.csv') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Bibliotecas para visualização dos dados
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import plotly.express as px

**Plotando a população estimada por capital**

In [ ]:
grafico = px.histogram(states_df, x="Capital", color="População estimada")
grafico.show()